In [0]:
dbutils.widgets.text("SinkPath", "", "Enter Sink Path")
sinkPath = dbutils.widgets.get("SinkPath")
print(sinkPath)

In [0]:
from pyspark.sql.functions import col, expr, from_json, DataFrame, sha2, concat_ws, lit
from pyspark.sql.types import *

In [0]:
# storage_account = "storagesudarshan3"
# container = "rawdata"

# sas_token = "sp=racwdlme&st=2025-08-08T10:21:19Z&se=2025-08-10T18:36:19Z&sv=2024-11-04&sr=c&sig=Q3kxqEdD%2BudMyDOhDn6ltT5OW229GA3Ts4IDdTiMa60%3D"

# spark.conf.set(f"fs.azure.sas.{container}.{storage_account}.blob.core.windows.net", sas_token)
# delta_path = f"wasbs://{container}@{storage_account}.blob.core.windows.net/{sinkPath}"

In [0]:
# bronze_customer_df = spark.read.csv(delta_path)
# display(bronze_customer_df)

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
# Configuration for Azure Blob Storage SAS token
storage_account = "sudarshanstorage3"
container = "rawdata"
sas_token = "sp=racwdlme&st=2025-08-08T10:21:19Z&se=2025-08-10T18:36:19Z&sv=2024-11-04&sr=c&sig=Q3kxqEdD%2BudMyDOhDn6ltT5OW229GA3Ts4IDdTiMa60%3D"

# Set SAS token for Azure Blob Storage container
spark.conf.set(f"fs.azure.sas.{container}.{storage_account}.blob.core.windows.net", sas_token)

# Specify the directory or folder path in your container
# sinkPath = "bronzelayer"  # You can modify this to your actual folder path
# List files in the container (without specifying folder path) to check the structure

dbutils.fs.ls(f"wasbs://{container}@{storage_account}.blob.core.windows.net/")
print('-----------------------------------------------------------------------------------')
# Delta path to be accessed
delta_path = f"wasbs://{container}@{storage_account}.blob.core.windows.net/{sinkPath}"

# List the files in the specified path
files = dbutils.fs.ls(delta_path)

# Print out the contents of the path
for file in files:
    print(f"File name: {file.name}, File path: {file.path}")

In [0]:
cust_path =f"{delta_path}bronzelayer/Customers"
pol_path =f"{delta_path}bronzelayer/Policies"

cust_df = spark.read.csv(cust_path)
pol_df = spark.read.csv(pol_path)
cust_df.printSchema()
pol_df.printSchema()

In [0]:
no_dup_cust_df = cust_df.dropDuplicates()
no_dup_pol_df = pol_df.dropDuplicates()

silver_path = f"{delta_path}Silver"

no_dup_cust_df.write.mode("overwrite").format("delta").save(silver_path)
no_dup_pol_df.write.mode("overwrite").format("delta").save(silver_path)

In [0]:
cust_silver = spark.read.format("delta").load(silver_path)
pol_silver = spark.read.format("delta").load(silver_path)

def mask_pii(df: DataFrame, pii_columns: dict) -> DataFrame:
    columns = df.columns

    for index, mask_type in pii_columns.items():
        if index >= len(columns):
            continue  

        col_name = columns[index]
        if col_name not in df.columns:
            continue

        if mask_type == "hash":
            df = df.withColumn(col_name, sha2(col(col_name).cast("string"), 256))
        elif mask_type == "redact":
            df = df.withColumn(col_name, lit("REDACTED"))
        else:
            raise ValueError(f"Unsupported mask type: '{mask_type}' for column '{col_name}'")

    return df

pii_columns = {
    1: "hash",   
    2: "hash",  
    3: "redact", 
    4: "redact"
}

masked_customers_df = mask_pii(cust_silver, pii_columns)
masked_customers_df.show(5)


gold_path = f"{delta_path}Gold"

masked_customers_df.write.mode("overwrite").format("delta").save(gold_path)